# Preprocesamiento del txt

El dataset se exporta de la placa ESP32 como un txt, que debe sufrir algunas correcciones antes de poder ser usado.

1. Agregar nombres de las columnas.
2. Eliminar datos de prueba.
3. Agregar horarios cruzando los datos de encendido del dispositivo con los de inicio de toma de datos.
4. 

In [ ]:
import pandas as pd
from datetime import datetime

path = './sensor_log_output_25_04.txt'
columns=['millis', 'humidity', 'light', 'temperature']

df = pd.read_csv(path, sep=',', header=None, names=columns)

Para eliminar los valores de prueba, eliminamos las entradas que difieren con la siguiente en menos de 2 segundos, o que tienen tiempo nulo.

In [ ]:
# Convert millis to integer
df['millis'] = pd.to_numeric(df['millis'], errors='coerce').astype('Int64')

differences = df['millis'].diff()

mask = (differences.isna()) | (differences.between(-2000, 2000))
df = df[~mask]

df.head()


In [ ]:
df = df.reset_index(drop=True)

indexes = df[df['millis'] < df['millis'].shift(1)].index
indexes = indexes.insert(0, 0)

print(indexes)

start_time_path = './start_times.csv'
start_times = pd.read_csv(start_time_path, sep=',', header=0)

for i in range(len(indexes) - 1):
    start_idx, end_idx = indexes[i], indexes[i + 1]
    start_date = datetime(2025, start_times.iloc[i]['month'], start_times.iloc[i]['day'], 
                          start_times.iloc[i]['hour'], start_times.iloc[i]['minute'])
    start_time = start_date.timestamp()
    df.loc[start_idx:end_idx, 'timestamp'] = df.loc[start_idx:end_idx, 'millis'].apply(lambda x: datetime.fromtimestamp(start_time + x / 1000))


In [ ]:
df.head()

df.to_csv('datos_planta.csv', index=False)